# 🎯 Fantasy Football Draft Preparation

Strategic analysis and draft prep tools to maximize your draft performance.

## Features:
- 📊 Positional Value Analysis
- 🎪 Tier-Based Rankings 
- 🔄 Mock Draft Scenarios
- 📈 ADP vs Projections
- 🎯 Draft Strategy Optimization
- ⚠️ Sleeper & Bust Identification

In [8]:
import pandas as pd
import numpy as np
import yaml
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.stats import zscore
import warnings
warnings.filterwarnings('ignore')

# Load data
with open('config/league-config.yaml', 'r') as f:
    config = yaml.safe_load(f)

cheat_sheet = pd.read_csv('draft_cheat_sheet.csv')
rankings = pd.read_csv('data/rankings_20250814.csv')

print("🏈 Draft Preparation Suite Loaded")
print(f"📊 {len(cheat_sheet)} players analyzed")
print(f"🏆 {config['league_name']} - {config['basic_settings']['teams']} teams")

🏈 Draft Preparation Suite Loaded
📊 50 players analyzed
🏆 Fantasy 24-25 - 14 teams


In [9]:
# Positional Value Analysis
def analyze_positional_value():
    """Analyze value by position and create tiers"""
    
    # Calculate positional rankings and tiers
    position_analysis = {}
    
    for position in ['QB', 'RB', 'WR', 'TE']:
        pos_players = cheat_sheet[cheat_sheet['Position'] == position].copy()
        pos_players = pos_players.sort_values('Custom_VBD', ascending=False)
        
        # Create tier breaks using k-means clustering on VBD
        if len(pos_players) > 1:
            n_tiers = min(6, len(pos_players))  # Max 6 tiers
            kmeans = KMeans(n_clusters=n_tiers, random_state=42, n_init=10)
            pos_players['Tier'] = kmeans.fit_predict(pos_players[['Custom_VBD']])
            
            # Sort tiers by VBD (highest VBD = Tier 1)
            tier_means = pos_players.groupby('Tier')['Custom_VBD'].mean().sort_values(ascending=False)
            tier_mapping = {old_tier: new_tier + 1 for new_tier, old_tier in enumerate(tier_means.index)}
            pos_players['Tier'] = pos_players['Tier'].map(tier_mapping)
        else:
            pos_players['Tier'] = 1
            
        pos_players['Pos_Rank'] = range(1, len(pos_players) + 1)
        position_analysis[position] = pos_players
    
    return position_analysis

position_data = analyze_positional_value()

# Display tier analysis
print("🎪 POSITIONAL TIER ANALYSIS\n")
for pos, data in position_data.items():
    print(f"\n{pos} TIERS:")
    tier_summary = data.groupby('Tier').agg({
        'Player': 'count',
        'Custom_VBD': ['mean', 'min', 'max']
    }).round(1)
    
    for tier in sorted(data['Tier'].unique()):
        tier_players = data[data['Tier'] == tier]
        avg_vbd = tier_players['Custom_VBD'].mean()
        count = len(tier_players)
        top_player = tier_players.iloc[0]['Player']
        print(f"  Tier {tier}: {count} players (avg VBD: {avg_vbd:.1f}) - led by {top_player}")

🎪 POSITIONAL TIER ANALYSIS


QB TIERS:
  Tier 1: 1 players (avg VBD: 91.7) - led by Lamar Jackson
  Tier 2: 1 players (avg VBD: 88.2) - led by Josh Allen
  Tier 3: 2 players (avg VBD: 68.0) - led by Jalen Hurts
  Tier 4: 1 players (avg VBD: 55.2) - led by Joe Burrow
  Tier 5: 1 players (avg VBD: 35.3) - led by Patrick Mahomes
  Tier 6: 2 players (avg VBD: 29.7) - led by Kyler Murray

RB TIERS:
  Tier 1: 3 players (avg VBD: 127.9) - led by Saquon Barkley
  Tier 2: 3 players (avg VBD: 95.1) - led by Christian McCaffrey
  Tier 3: 2 players (avg VBD: 77.9) - led by Josh Jacobs
  Tier 4: 4 players (avg VBD: 64.5) - led by Kyren Williams
  Tier 5: 4 players (avg VBD: 46.9) - led by James Cook
  Tier 6: 3 players (avg VBD: 36.8) - led by Kenneth Walker III

WR TIERS:
  Tier 1: 1 players (avg VBD: 120.6) - led by Ja'Marr Chase
  Tier 2: 1 players (avg VBD: 88.3) - led by Justin Jefferson
  Tier 3: 2 players (avg VBD: 71.1) - led by CeeDee Lamb
  Tier 4: 4 players (avg VBD: 56.3) - led by Amon-

In [10]:
# Positional Scarcity Analysis
def create_scarcity_analysis():
    """Analyze positional scarcity and draft urgency"""
    
    # Calculate players needed vs available
    num_teams = config['basic_settings']['teams']
    roster_slots = config['roster']['roster_slots']
    
    scarcity_data = []
    
    for pos in ['QB', 'RB', 'WR', 'TE']:
        if pos in roster_slots:
            # Required starters
            starters_needed = roster_slots[pos] * num_teams
            
            # Add FLEX eligibility
            if pos in ['RB', 'WR', 'TE']:
                flex_spots = roster_slots.get('FLEX', 0) * num_teams
                total_needed = starters_needed + flex_spots
            else:
                total_needed = starters_needed
            
            # Available players (using top 200 as "draftable")
            pos_players = cheat_sheet[cheat_sheet['Position'] == pos]
            top_available = len(pos_players[pos_players['Draft_Rank'] <= 200])
            
            scarcity_ratio = total_needed / max(top_available, 1)
            
            scarcity_data.append({
                'Position': pos,
                'Starters_Needed': starters_needed,
                'Total_Needed': total_needed,
                'Top_Available': top_available,
                'Scarcity_Ratio': scarcity_ratio,
                'Urgency': 'HIGH' if scarcity_ratio > 0.8 else 'MEDIUM' if scarcity_ratio > 0.5 else 'LOW'
            })
    
    scarcity_df = pd.DataFrame(scarcity_data)
    
    # Visualization
    fig = go.Figure()
    
    colors = {'HIGH': '#FF6B6B', 'MEDIUM': '#FFD93D', 'LOW': '#6BCF7F'}
    
    fig.add_trace(go.Bar(
        x=scarcity_df['Position'],
        y=scarcity_df['Scarcity_Ratio'],
        marker_color=[colors[urgency] for urgency in scarcity_df['Urgency']],
        text=[f"{ratio:.2f}<br>{urgency}" for ratio, urgency in 
              zip(scarcity_df['Scarcity_Ratio'], scarcity_df['Urgency'])],
        textposition='auto'
    ))
    
    fig.update_layout(
        title="🎯 Positional Scarcity Analysis",
        xaxis_title="Position",
        yaxis_title="Scarcity Ratio (Needed/Available)",
        height=400
    )
    
    fig.show()
    
    print("\n📊 SCARCITY SUMMARY:")
    display(HTML(scarcity_df.to_html(index=False, classes='table table-striped')))
    
    return scarcity_df

scarcity_analysis = create_scarcity_analysis()


📊 SCARCITY SUMMARY:


Position,Starters_Needed,Total_Needed,Top_Available,Scarcity_Ratio,Urgency
QB,14,14,8,1.750000,HIGH
RB,28,42,19,2.210526,HIGH
WR,28,42,16,2.625000,HIGH
TE,14,28,7,4.000000,HIGH


In [11]:
# Value vs ADP Analysis (Sleepers & Busts)
def find_sleepers_and_busts():
    """Identify players with value discrepancies vs ADP"""
    
    # Calculate ADP vs Value differential
    analysis_df = cheat_sheet.copy()
    analysis_df['ADP_vs_Rank'] = analysis_df['ECR'] - analysis_df['Draft_Rank']
    
    # Z-score for identifying outliers
    analysis_df['VBD_zscore'] = zscore(analysis_df['Custom_VBD'])
    analysis_df['ADP_zscore'] = zscore(analysis_df['ECR'])
    analysis_df['Value_Diff'] = analysis_df['VBD_zscore'] - analysis_df['ADP_zscore']
    
    # Categorize players
    def categorize_player(row):
        if row['Value_Diff'] > 0.75:  # High VBD, low ADP
            return 'SLEEPER'
        elif row['Value_Diff'] < -0.75:  # Low VBD, high ADP
            return 'BUST_RISK'
        elif abs(row['Value_Diff']) <= 0.25:
            return 'CONSENSUS'
        else:
            return 'SLIGHT_VALUE' if row['Value_Diff'] > 0 else 'SLIGHT_RISK'
    
    analysis_df['Category'] = analysis_df.apply(categorize_player, axis=1)
    
    # Create visualization
    fig = px.scatter(
        analysis_df,
        x='ECR',
        y='Custom_VBD',
        color='Category',
        hover_data=['Player', 'Position', 'Team'],
        title="🎯 Sleepers vs Bust Risks (VBD vs ADP)",
        labels={'ECR': 'Expert Consensus Ranking (ADP)', 'Custom_VBD': 'Value Above Replacement'}
    )
    
    fig.update_traces(marker=dict(size=8))
    fig.update_layout(height=600)
    fig.show()
    
    # Top sleepers and bust risks
    print("\n🔥 TOP SLEEPER PICKS (High Value, Low ADP):")
    sleepers = analysis_df[analysis_df['Category'] == 'SLEEPER'].sort_values('Value_Diff', ascending=False).head(10)
    for _, player in sleepers.iterrows():
        print(f"  {player['Player']:<25} {player['Position']:<3} (ADP: {player['ECR']:>3.0f}, VBD: {player['Custom_VBD']:>6.1f})")
    
    print("\n⚠️ BUST RISK ALERTS (Low Value, High ADP):")
    busts = analysis_df[analysis_df['Category'] == 'BUST_RISK'].sort_values('Value_Diff').head(10)
    for _, player in busts.iterrows():
        print(f"  {player['Player']:<25} {player['Position']:<3} (ADP: {player['ECR']:>3.0f}, VBD: {player['Custom_VBD']:>6.1f})")
    
    return analysis_df

value_analysis = find_sleepers_and_busts()


🔥 TOP SLEEPER PICKS (High Value, Low ADP):
  Saquon Barkley            RB  (ADP:   4, VBD:  131.3)
  Bijan Robinson            RB  (ADP:   1, VBD:  127.3)
  Jahmyr Gibbs              RB  (ADP:   3, VBD:  125.0)
  Ja'Marr Chase             WR  (ADP:   6, VBD:  120.6)
  Justin Jefferson          WR  (ADP:   2, VBD:   88.3)
  Christian McCaffrey       RB  (ADP:  13, VBD:   96.9)
  Derrick Henry             RB  (ADP:  14, VBD:   95.8)
  De'Von Achane             RB  (ADP:  15, VBD:   92.7)
  CeeDee Lamb               WR  (ADP:   5, VBD:   74.4)
  Brock Bowers              TE  (ADP:  18, VBD:   85.4)

⚠️ BUST RISK ALERTS (Low Value, High ADP):
  Kyler Murray              QB  (ADP:  79, VBD:   30.0)
  Travis Kelce              TE  (ADP:  80, VBD:   35.5)
  Mark Andrews              TE  (ADP:  78, VBD:   34.6)
  Baker Mayfield            QB  (ADP:  69, VBD:   29.3)
  T.J. Hockenson            TE  (ADP:  70, VBD:   33.5)
  Patrick Mahomes           QB  (ADP:  64, VBD:   35.3)
  Sam LaPorta   

In [12]:
# Mock Draft Simulator
class MockDraftSimulator:
    def __init__(self, config, cheat_sheet, draft_position):
        self.config = config
        self.cheat_sheet = cheat_sheet.copy()
        self.num_teams = config['basic_settings']['teams']
        self.draft_position = draft_position
        self.rounds = 16  # Full draft
        
    def simulate_pick(self, available_players, team_needs, pick_position):
        """Simulate how other teams might pick"""
        
        # Weight by need and value
        weights = []
        for _, player in available_players.iterrows():
            pos = player['Position']
            base_value = player['Custom_VBD']
            
            # Need multiplier
            need_multiplier = team_needs.get(pos, 1)
            
            # Add some randomness (other teams don't draft perfectly)
            randomness = np.random.normal(1, 0.15)
            
            final_weight = base_value * need_multiplier * randomness
            weights.append(max(0, final_weight))
        
        # Select based on weighted probabilities
        if weights and sum(weights) > 0:
            probs = np.array(weights) / sum(weights)
            choice_idx = np.random.choice(len(available_players), p=probs)
            return available_players.iloc[choice_idx]
        else:
            return available_players.iloc[0] if len(available_players) > 0 else None
    
    def run_mock_draft(self, strategy='BEST_AVAILABLE'):
        """Run a complete mock draft simulation"""
        
        available = self.cheat_sheet[self.cheat_sheet['Draft_Rank'] <= 250].copy()
        my_team = []
        draft_log = []
        
        # Simple team needs tracking
        team_needs = {i: {'QB': 1.5, 'RB': 2.0, 'WR': 2.0, 'TE': 1.2, 'DEF': 0.5, 'K': 0.3} 
                     for i in range(self.num_teams)}
        
        for round_num in range(1, self.rounds + 1):
            # Determine pick order (snake draft)
            if round_num % 2 == 1:
                pick_order = list(range(self.num_teams))
            else:
                pick_order = list(range(self.num_teams - 1, -1, -1))
            
            for pick_in_round, team_idx in enumerate(pick_order):
                if len(available) == 0:
                    break
                
                overall_pick = (round_num - 1) * self.num_teams + pick_in_round + 1
                
                if team_idx == self.draft_position - 1:  # My pick
                    # Apply strategy
                    if strategy == 'BEST_AVAILABLE':
                        my_pick = available.iloc[0]
                    elif strategy == 'POSITIONAL_NEED':
                        # Prioritize positions we need
                        pos_priority = ['RB', 'WR', 'QB', 'TE', 'DEF', 'K']
                        my_pick = None
                        for pos in pos_priority:
                            pos_available = available[available['Position'] == pos]
                            if len(pos_available) > 0 and team_needs[team_idx][pos] > 0.8:
                                my_pick = pos_available.iloc[0]
                                break
                        if my_pick is None:
                            my_pick = available.iloc[0]
                    else:
                        my_pick = available.iloc[0]
                    
                    my_team.append(my_pick.to_dict())
                    selected_player = my_pick
                    
                else:  # Other team's pick
                    selected_player = self.simulate_pick(available, team_needs[team_idx], team_idx)
                
                if selected_player is not None:
                    # Remove from available
                    available = available[available['Player'] != selected_player['Player']]
                    
                    # Update team needs
                    pos = selected_player['Position']
                    if pos in team_needs[team_idx]:
                        team_needs[team_idx][pos] *= 0.7  # Reduce need
                    
                    # Log pick
                    draft_log.append({
                        'Pick': overall_pick,
                        'Round': round_num,
                        'Team': team_idx + 1,
                        'MyPick': team_idx == self.draft_position - 1,
                        'Player': selected_player['Player'],
                        'Position': selected_player['Position'],
                        'VBD': selected_player['Custom_VBD']
                    })
        
        return my_team, draft_log

# Mock Draft Interface
position_selector = widgets.IntSlider(
    value=1, min=1, max=config['basic_settings']['teams'],
    description='Draft Position:'
)

strategy_selector = widgets.Dropdown(
    options=['BEST_AVAILABLE', 'POSITIONAL_NEED'],
    value='BEST_AVAILABLE',
    description='Strategy:'
)

mock_button = widgets.Button(
    description='🎲 Run Mock Draft',
    button_style='primary'
)

mock_output = widgets.Output()

def run_mock(button):
    with mock_output:
        clear_output()
        print(f"🎯 Running mock draft from position {position_selector.value} with {strategy_selector.value} strategy...\n")
        
        simulator = MockDraftSimulator(config, cheat_sheet, position_selector.value)
        my_team, draft_log = simulator.run_mock_draft(strategy_selector.value)
        
        print("🏆 YOUR MOCK DRAFT TEAM:\n")
        total_vbd = 0
        for i, player in enumerate(my_team, 1):
            round_num = ((i - 1) // 1) + 1
            print(f"R{round_num:2d}: {player['Player']:<25} {player['Position']:<3} (VBD: {player['Custom_VBD']:>6.1f})")
            total_vbd += player['Custom_VBD']
        
        print(f"\n📊 Total Team VBD: {total_vbd:.1f}")
        
        # Position breakdown
        pos_counts = {}
        for player in my_team:
            pos = player['Position']
            pos_counts[pos] = pos_counts.get(pos, 0) + 1
        
        print("\n📋 Roster Composition:")
        for pos, count in sorted(pos_counts.items()):
            print(f"  {pos}: {count}")

mock_button.on_click(run_mock)

print("🎲 MOCK DRAFT SIMULATOR")
display(widgets.VBox([
    widgets.HBox([position_selector, strategy_selector]),
    mock_button,
    mock_output
]))

🎲 MOCK DRAFT SIMULATOR


In [ ]:
# Draft Strategy Recommendations
def generate_strategy_guide(draft_position):
    """Generate position-specific draft strategy"""
    
    num_teams = config['basic_settings']['teams']
    
    strategies = {
        'early': (1, 3),
        'mid_early': (4, 6), 
        'middle': (7, 10),
        'mid_late': (11, 12),
        'late': (13, num_teams)
    }
    
    position_type = None
    for pos_name, (start, end) in strategies.items():
        if start <= draft_position <= end:
            position_type = pos_name
            break
    
    recommendations = {
        'early': {
            'rounds_1_2': "Target elite RB/WR. You get first pick at premium positions.",
            'rounds_3_4': "Continue RB/WR or grab top QB if available.",
            'mid_rounds': "Address positional needs, target value.",
            'late_rounds': "Handcuffs, upside plays, DEF/K"
        },
        'mid_early': {
            'rounds_1_2': "Best available RB/WR, maybe elite TE.",
            'rounds_3_4': "Round out RB/WR corps, consider QB.",
            'mid_rounds': "Fill positional needs, target breakouts.",
            'late_rounds': "Depth and lottery tickets."
        },
        'middle': {
            'rounds_1_2': "Balanced approach, target tier breaks.",
            'rounds_3_4': "Address team needs, consider positional runs.",
            'mid_rounds': "Value hunting, sleeper targets.",
            'late_rounds': "Handcuffs and upside."
        },
        'mid_late': {
            'rounds_1_2': "Target falling tier-1 players.", 
            'rounds_3_4': "Build solid foundation, avoid reaches.",
            'mid_rounds': "Value picks, positional depth.",
            'late_rounds': "Upside swings."
        },
        'late': {
            'rounds_1_2': "Back-to-back picks advantage! Target tier breaks.",
            'rounds_3_4': "Use turn to grab complementary pieces.",
            'mid_rounds': "Be aggressive on upside plays.",
            'late_rounds': "Lottery tickets and handcuffs."
        }
    }
    
    print(f"\n🎯 DRAFT STRATEGY FOR POSITION {draft_position} ({position_type.upper()})\n")
    
    if position_type in recommendations:
        for phase, advice in recommendations[position_type].items():
            print(f"📍 {phase.replace('_', ' ').title()}: {advice}")
    
    # Position-specific targets
    print(f"\n🎪 TARGET PLAYERS BY ROUND:\n")
    
    # Round 1 targets
    round1_picks = range((draft_position - 1), len(cheat_sheet), num_teams)
    round1_target = None
    for pick in round1_picks:
        if pick < len(cheat_sheet):
            round1_target = cheat_sheet.iloc[pick]
            break
    
    if round1_target is not None:
        print(f"Round 1 Target: {round1_target['Player']} ({round1_target['Position']})")
    
    # Show players likely available at your picks
    your_picks = []
    for round_num in range(1, 6):  # First 5 rounds
        if round_num % 2 == 1:  # Odd rounds
            pick_num = (round_num - 1) * num_teams + draft_position
        else:  # Even rounds (snake back)
            pick_num = round_num * num_teams - draft_position + 1
        your_picks.append((round_num, pick_num))
    
    print("\n📅 Your Pick Schedule:")
    for round_num, pick_num in your_picks:
        if pick_num <= len(cheat_sheet):
            target_range = cheat_sheet.iloc[max(0, pick_num-3):pick_num+2]
            print(f"Round {round_num} (Pick {pick_num}): Target range around {target_range.iloc[2]['Player'] if len(target_range) > 2 else 'N/A'}")

# Interactive strategy guide
strategy_position = widgets.IntSlider(
    value=1, min=1, max=config['basic_settings']['teams'],
    description='Your Draft Position:'
)

strategy_button = widgets.Button(
    description='📋 Get Strategy Guide',
    button_style='info'
)

strategy_output = widgets.Output()

def show_strategy(button):
    with strategy_output:
        clear_output()
        generate_strategy_guide(strategy_position.value)

strategy_button.on_click(show_strategy)

display(widgets.VBox([
    strategy_position,
    strategy_button,
    strategy_output
]))

## 🎯 Draft Preparation Summary

### Key Tools Created:

1. **📊 Positional Value Analysis**
   - Tier-based rankings using clustering
   - Identifies natural breakpoints

2. **🎪 Scarcity Analysis** 
   - Shows which positions to prioritize
   - Urgency ratings based on supply/demand

3. **🔥 Sleeper/Bust Identification**
   - Value vs ADP discrepancies
   - Target undervalued players

4. **🎲 Mock Draft Simulator**
   - Test different strategies
   - Practice from your draft position

5. **📋 Position-Specific Strategy**
   - Tailored advice for your draft slot
   - Round-by-round guidance

### Pre-Draft Checklist:
- ✅ Run scarcity analysis to identify urgent positions
- ✅ Review sleeper/bust lists for value targets
- ✅ Simulate 3-5 mock drafts from your position
- ✅ Study tier breaks for your target rounds
- ✅ Plan contingency strategies for different scenarios

**Now you're ready to dominate your draft! 🏆**